In [2]:
from ultralytics import YOLO

In [3]:
model_file = 'runs/detect/star2_up-yolo11n_1000/weights/best.pt'
model = YOLO(f"{model_file}")

In [ ]:
model.benchmark(data='coco.yaml', imgsz=640, format='-')

Setup complete ✅ (24 CPUs, 125.6 GB RAM, 769.6/1832.2 GB disk)

Benchmarks complete for runs/detect/star2_up-yolo11n_1000/weights/best.pt on coco.yaml at imgsz=640 (47.39s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
    Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)     FPS
0  PyTorch       ✅        5.4               0.4006                   2.218  450.65



,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im),FPS
0,PyTorch,✅,5.4,0.4006,2.218,450.65


In [20]:
result = model.benchmark(data='coco.yaml', imgsz=640, format='engine',half=False, int8=False, device='cuda')

Setup complete ✅ (24 CPUs, 125.6 GB RAM, 770.7/1832.2 GB disk)

Benchmarks complete for runs/detect/star2_up-yolo11n_1000/weights/best.pt on coco.yaml at imgsz=640 (85.87s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
     Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)      FPS
0  TensorRT       ✅       16.8               0.4008                  0.9257  1079.13



In [23]:
import numpy as np
values = []
values += [result.values[0][3:]]
values += [result.values[0][3:]]
values += [result.values[0][3:]]
values += [result.values[0][3:]]
values += [result.values[0][3:]]
values = np.array(sorted(values, key=lambda x : x[1])[1:-1]).T

In [25]:
values[0]

array([0.4008, 0.4008, 0.4008], dtype=object)

In [5]:
model_file = 'yolo11n.pt'
model = YOLO(f"{model_file}")

In [6]:
model.benchmark(data='coco.yaml', imgsz=640, format='-')

Setup complete ✅ (24 CPUs, 125.6 GB RAM, 770.9/1832.2 GB disk)

Benchmarks complete for yolo11n.pt on coco.yaml at imgsz=640 (48.36s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
    Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)     FPS
0  PyTorch       ✅        5.4               0.3923                  2.3151  431.75



,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im),FPS
0,PyTorch,✅,5.4,0.3923,2.3151,431.75


In [7]:
model.benchmark(data='coco.yaml', imgsz=640, format='engine')

Setup complete ✅ (24 CPUs, 125.6 GB RAM, 771.0/1832.2 GB disk)

Benchmarks complete for yolo11n.pt on coco.yaml at imgsz=640 (84.98s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
     Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)      FPS
0  TensorRT       ✅       17.0               0.3913                  0.9095  1098.27



,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im),FPS
0,TensorRT,✅,17.0,0.3913,0.9095,1098.27
